## <div align="center">  Data Analysis Project </div>
#### <div align="center"> Semester Ganjil 2023/2024 </div>
---

In [ ]:
import datetime
import uuid

studentName = "Marcell Nathan Santoso"
studentNIM = "00000092351"
studentClass = "EL"

In [ ]:
myDate = datetime.datetime.now()
myDevice = str(uuid.uuid1())

print("Name: \t\t{}".format(studentName))
print("NIM: \t\t{}".format(studentNIM))
print("student Class: \t\t{}".format(studentClass))
print("Start: \t\t{}".format(myDate))
print("Device ID: \t{}".format(myDevice))

---

## <div align="center"> Our Project Code  </div>

In [ ]:
# Import dasar
import os                  # Untuk operasi sistem seperti navigasi path file dan direktori
import numpy as np         # Untuk komputasi numerik dan operasi array/matriks
import pandas as pd        # Untuk manipulasi dan analisis data dalam bentuk DataFrame
import random 
from PIL import Image, ImageOps     # Untuk membuka, memanipulasi, dan menyimpan berbagai format gambar
from PIL import ImageFilter

# Import untuk DICOM
import pydicom            # Untuk membaca dan mengolah file DICOM (format gambar medis)
from pydicom.pixel_data_handlers.util import apply_voi_lut  # Untuk mengatur kontras gambar DICOM

# Import sklearn
from sklearn.model_selection import train_test_split    # Untuk membagi data menjadi set training dan testing
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay  # Untuk evaluasi model
from sklearn.preprocessing import LabelEncoder          # Untuk mengubah label kategorikal menjadi numerik
from sklearn.ensemble import RandomForestClassifier    # Algoritma klasifikasi Random Forest
from sklearn.naive_bayes import GaussianNB             # Algoritma klasifikasi Naive Bayes
from sklearn.neighbors import KNeighborsClassifier     # Algoritma klasifikasi K-Nearest Neighbors
from sklearn.svm import SVC                           # Algoritma klasifikasi Support Vector Machine
import xgboost as xgb # Algoritma klasifikasi XGBoost
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import RocCurveDisplay

# Import untuk visualisasi
import matplotlib.pyplot as plt  # Untuk membuat plot dan visualisasi data
import seaborn as sns           # Untuk membuat visualisasi statistik yang lebih menarik

# Import TensorFlow dan Keras
import tensorflow as tf                               # Framework untuk deep learning
from tensorflow.keras.models import Sequential        # Untuk membuat model neural network sekuensial
from tensorflow.keras.layers import Conv2D           # Layer konvolusi 2D untuk CNN
from tensorflow.keras.layers import MaxPooling2D     # Layer pooling untuk CNN
from tensorflow.keras.layers import Flatten          # Layer untuk mengubah data multidimensi menjadi 1D
from tensorflow.keras.layers import Dense            # Layer fully connected untuk neural network
from tensorflow.keras.layers import Dropout          # Layer untuk mencegah overfitting
from tensorflow.keras.utils import to_categorical    # Untuk mengubah label menjadi format one-hot encoding
from tensorflow.keras.preprocessing.image import load_img, img_to_array

### Reading Data

In [ ]:
    extracted_folder = 'brain_dataset'  # Mendefinisikan nama folder tempat dataset gambar otak berada
    csv_path = os.path.join(extracted_folder, 'ct_brain.csv')  # Membuat path lengkap ke file CSV yang berisi metadata dataset
    ct_brain_data = pd.read_csv(csv_path)  # Membaca file CSV yang berisi data metadata ke dalam DataFrame pandas
    image_size = (64, 64)  # Mendefinisikan ukuran gambar yang diinginkan untuk preprocessing (dalam hal ini 64x64 piksel)

In [ ]:
# Untuk Melihat ada berapa row di dataset tersebut
ct_brain_data

In [ ]:
# Memberikan informasi pada dataset
ct_brain_data.info()

In [ ]:
# Melihat apakah adanya data yg null atau tidak
ct_brain_data.isnull().sum()

### Visualization 

In [ ]:
folder_path = 'brain_dataset'

# Daftar kategori penyakit
categories = ['cancer', 'tumor', 'aneurysm']

# Menghitung jumlah gambar di tiap kategori
category_counts = {'cancer': 0, 'tumor': 0, 'aneurysm': 0}

for category in categories:
    category_path = os.path.join(folder_path, category)
    if os.path.exists(category_path):
        # Menghitung gambar dalam folder kategori
        category_counts[category] = len([f for f in os.listdir(category_path) if f.endswith(('.jpg', '.png'))])
    else:
        category_counts[category] = 0

# Cek jika semua kategori ada dan tidak kosong
category_counts = {key: value for key, value in category_counts.items() if value > 0}

# Jika semua kategori kosong, keluar dari program
if not category_counts:
    print("Tidak ada gambar untuk diproses!")
else:
    # Membuat pie chart
    plt.figure(figsize=(6, 6))
    plt.pie(category_counts.values(), labels=category_counts.keys(), autopct='%1.1f%%', startangle=90, colors=['#ff9999','#66b3ff','#99ff99'])
    plt.title('Distribution of Diseases: Cancer, Aneurysm, Tumor')
    plt.axis('equal')  # Membuat pie chart menjadi lingkaran sempurna
    plt.show()

    # Menampilkan jumlah gambar di setiap kategori
    print(category_counts)


In [ ]:
# Menghitung ukuran gambar
widths = []
heights = []

for category in categories:
    category_path = os.path.join(folder_path, category)
    if os.path.exists(category_path):
        for image_name in os.listdir(category_path):
            if image_name.endswith(('.jpg', '.png')):
                image_path = os.path.join(category_path, image_name)
                with Image.open(image_path) as img:
                    widths.append(img.width)
                    heights.append(img.height)

# Membuat histogram untuk ukuran gambar
plt.figure(figsize=(12, 6))

# Histogram untuk lebar gambar
plt.subplot(1, 2, 1)
plt.hist(widths, bins=20, color='blue', edgecolor='black')
plt.title('Histogram of Image Widths')
plt.xlabel('Width (pixels)')
plt.ylabel('Frequency')

# Histogram untuk tinggi gambar
plt.subplot(1, 2, 2)
plt.hist(heights, bins=20, color='green', edgecolor='black')
plt.title('Histogram of Image Heights')
plt.xlabel('Height (pixels)')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:

# Path folder dataset
folder_path = 'brain_dataset'

# Daftar kategori penyakit
categories = ['cancer', 'tumor', 'aneurysm']

# Fungsi untuk menampilkan visualisasi RGB dari gambar
def plot_rgb_histogram(image_path):
    # Membaca gambar
    img = Image.open(image_path)
    
    # Mengonversi gambar ke dalam format RGB
    img_rgb = img.convert('RGB')

    # Mendapatkan komponen RGB
    r, g, b = img_rgb.split()

    # Mengubah komponen menjadi numpy array untuk analisis lebih lanjut
    r = np.array(r)
    g = np.array(g)
    b = np.array(b)

    # Membuat histogram untuk masing-masing saluran warna
    plt.figure(figsize=(12, 6))

    # Histogram untuk Red channel
    plt.subplot(1, 3, 1)
    plt.hist(r.flatten(), bins=256, color='red', alpha=0.6)
    plt.title('Histogram Red')

    # Histogram untuk Green channel
    plt.subplot(1, 3, 2)
    plt.hist(g.flatten(), bins=256, color='green', alpha=0.6)
    plt.title('Histogram Green')

    # Histogram untuk Blue channel
    plt.subplot(1, 3, 3)
    plt.hist(b.flatten(), bins=256, color='blue', alpha=0.6)
    plt.title('Histogram Blue')

    plt.tight_layout()
    plt.show()

    # Menampilkan gambar RGB
    plt.figure(figsize=(6, 6))
    plt.imshow(img_rgb)
    plt.axis('off')  # Menyembunyikan axis
    plt.title(f'Gambar RGB: {image_path}')
    plt.show()

# Menghitung jumlah gambar di tiap kategori dan memilih gambar acak
category_counts = {'cancer': 0, 'tumor': 0, 'aneurysm': 0}

# Memilih gambar secara acak dari setiap kategori untuk visualisasi
sample_images = []

for category in categories:
    category_path = os.path.join(folder_path, category)
    if os.path.exists(category_path):
        images = [f for f in os.listdir(category_path) if f.endswith(('.jpg', '.png'))]
        category_counts[category] = len(images)
        
        # Memilih satu gambar acak dari folder kategori
        if images:
            random_image = random.choice(images)
            sample_images.append(os.path.join(category_path, random_image))
    else:
        category_counts[category] = 0

# Menampilkan jumlah gambar di setiap kategori
print(category_counts)

# Menampilkan RGB histogram untuk gambar yang dipilih secara acak dari setiap kategori
for image_path in sample_images:
    plot_rgb_histogram(image_path)

In [ ]:
# Path folder dataset
folder_path = 'brain_dataset'

# Daftar kategori penyakit
categories = ['cancer', 'tumor', 'aneurysm']

# Fungsi untuk menampilkan visualisasi RGB lengkap
def visualize_rgb_statistics(image_path, category):
    # Membaca gambar
    img = Image.open(image_path).convert('RGB')  # Konversi gambar ke RGB
    r, g, b = img.split()  # Memisahkan saluran RGB
    
    # Mengubah saluran warna menjadi numpy array
    r_array = np.array(r)
    g_array = np.array(g)
    b_array = np.array(b)
    
    # Fungsi untuk menghitung statistik
    def calculate_statistics(channel_array):
        return {
            "mean": np.mean(channel_array),
            "std_dev": np.std(channel_array),
            "min": np.min(channel_array),
            "max": np.max(channel_array),
        }

    # Hitung statistik untuk masing-masing saluran warna
    r_stats = calculate_statistics(r_array)
    g_stats = calculate_statistics(g_array)
    b_stats = calculate_statistics(b_array)
    
    # Membuat visualisasi
    fig, axes = plt.subplots(2, 4, figsize=(18, 10))
    
    # Tampilkan gambar RGB
    axes[0, 0].imshow(img)
    axes[0, 0].axis('off')
    axes[0, 0].set_title(f'{category.capitalize()} Image converted to RGB')
    
    # Tampilkan masing-masing saluran warna
    axes[0, 1].imshow(r, cmap='Reds')
    axes[0, 1].axis('off')
    axes[0, 1].set_title('RGB Red channel')
    
    axes[0, 2].imshow(g, cmap='Greens')
    axes[0, 2].axis('off')
    axes[0, 2].set_title('RGB Green channel')
    
    axes[0, 3].imshow(b, cmap='Blues')
    axes[0, 3].axis('off')
    axes[0, 3].set_title('RGB Blue channel')
    
    # Histogram Red channel
    axes[1, 1].hist(r_array.flatten(), bins=256, color='red', alpha=0.7)
    axes[1, 1].set_title(f"Red Histogram\nMean: {r_stats['mean']:.1f}, Std.dev: {r_stats['std_dev']:.1f}")
    axes[1, 1].set_xlim(0, 255)
    axes[1, 1].set_xlabel('Pixel value')
    axes[1, 1].set_ylabel('Count')
    
    # Histogram Green channel
    axes[1, 2].hist(g_array.flatten(), bins=256, color='green', alpha=0.7)
    axes[1, 2].set_title(f"Green Histogram\nMean: {g_stats['mean']:.1f}, Std.dev: {g_stats['std_dev']:.1f}")
    axes[1, 2].set_xlim(0, 255)
    axes[1, 2].set_xlabel('Pixel value')
    
    # Histogram Blue channel
    axes[1, 3].hist(b_array.flatten(), bins=256, color='blue', alpha=0.7)
    axes[1, 3].set_title(f"Blue Histogram\nMean: {b_stats['mean']:.1f}, Std.dev: {b_stats['std_dev']:.1f}")
    axes[1, 3].set_xlim(0, 255)
    axes[1, 3].set_xlabel('Pixel value')
    
    # Statistik keseluruhan
    axes[1, 0].axis('off')
    stats_text = (
        f"Mean: {r_stats['mean']:.1f}, {g_stats['mean']:.1f}, {b_stats['mean']:.1f}\n"
        f"Std.dev: {r_stats['std_dev']:.1f}, {g_stats['std_dev']:.1f}, {b_stats['std_dev']:.1f}\n"
        f"Min: {r_stats['min']}, {g_stats['min']}, {b_stats['min']}\n"
        f"Max: {r_stats['max']}, {g_stats['max']}, {b_stats['max']}"
    )
    axes[1, 0].text(0.5, 0.5, stats_text, ha='center', va='center', fontsize=12)
    axes[1, 0].set_title('Overall Statistics')
    
    plt.tight_layout()
    plt.show()

# Mengambil satu gambar acak dari setiap kategori
for category in categories:
    category_path = os.path.join(folder_path, category)
    if os.path.exists(category_path):
        # List gambar dalam kategori
        images = [f for f in os.listdir(category_path) if f.endswith(('.jpg', '.png'))]
        if images:
            # Memilih satu gambar acak
            random_image = random.choice(images)
            image_path = os.path.join(category_path, random_image)
            
            # Visualisasi RGB dari gambar tersebut
            visualize_rgb_statistics(image_path, category)


In [ ]:

folder_path = 'tumor'  # Mendefinisikan path folder tempat gambar 

# Membuat daftar file gambar yang memiliki ekstensi .png, .jpg, .jpeg, .bmp, .gif, atau .dcm (DICOM)
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.dcm'))]

# Jika tidak ada gambar yang ditemukan, mencetak pesan error
if not image_files:
    print("Tidak ada gambar yang ditemukan dalam folder.")
else:
    n_cols = 5  # Menentukan jumlah kolom pada tampilan gambar
    n_rows = (len(image_files) + n_cols - 1) // n_cols  # Menentukan jumlah baris gambar berdasarkan jumlah file gambar

    # Membuat subplots untuk menampilkan gambar-gambar dalam grid
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))

    axes = axes.flatten()  # Mengubah array 2D axes menjadi 1D agar lebih mudah diakses

    # Menampilkan gambar satu per satu
    for i, image_file in enumerate(image_files):
        img_path = os.path.join(folder_path, image_file)  # Mendapatkan path lengkap untuk setiap gambar

        # Jika gambar adalah file DICOM, memprosesnya sebagai gambar medis
        if img_path.lower().endswith('.dcm'):
            dicom_data = pydicom.dcmread(img_path)  # Membaca file DICOM
            img_array = apply_voi_lut(dicom_data.pixel_array, dicom_data)  # Mengaplikasikan VOI LUT pada data piksel

            # Jika gambar DICOM dalam format MONOCHROME1, mengonversi gambar menjadi format yang benar
            if dicom_data.PhotometricInterpretation == "MONOCHROME1":
                img_array = np.amax(img_array) - img_array

            axes[i].imshow(img_array, cmap='gray')  # Menampilkan gambar DICOM dalam grayscale

        else:
            # Jika bukan file DICOM, membuka gambar biasa menggunakan PIL
            img = Image.open(img_path)
            axes[i].imshow(img)  # Menampilkan gambar dengan format yang sesuai

        axes[i].axis('off')  # Menonaktifkan axis pada gambar
        axes[i].set_title(image_file)  # Menampilkan nama file gambar sebagai judul

    # Menonaktifkan axis pada subplot yang tidak terpakai
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()  # Menyesuaikan layout agar gambar tidak saling bertumpukan
    plt.show()  # Menampilkan grid gambar


In [ ]:

folder_path = 'cancer'  # Mendefinisikan path folder tempat gambar 

# Membuat daftar file gambar yang memiliki ekstensi .png, .jpg, .jpeg, .bmp, .gif, atau .dcm (DICOM)
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.dcm'))]

# Jika tidak ada gambar yang ditemukan, mencetak pesan error
if not image_files:
    print("Tidak ada gambar yang ditemukan dalam folder.")
else:
    n_cols = 5  # Menentukan jumlah kolom pada tampilan gambar
    n_rows = (len(image_files) + n_cols - 1) // n_cols  # Menentukan jumlah baris gambar berdasarkan jumlah file gambar

    # Membuat subplots untuk menampilkan gambar-gambar dalam grid
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))

    axes = axes.flatten()  # Mengubah array 2D axes menjadi 1D agar lebih mudah diakses

    # Menampilkan gambar satu per satu
    for i, image_file in enumerate(image_files):
        img_path = os.path.join(folder_path, image_file)  # Mendapatkan path lengkap untuk setiap gambar

        # Jika gambar adalah file DICOM, memprosesnya sebagai gambar medis
        if img_path.lower().endswith('.dcm'):
            dicom_data = pydicom.dcmread(img_path)  # Membaca file DICOM
            img_array = apply_voi_lut(dicom_data.pixel_array, dicom_data)  # Mengaplikasikan VOI LUT pada data piksel

            # Jika gambar DICOM dalam format MONOCHROME1, mengonversi gambar menjadi format yang benar
            if dicom_data.PhotometricInterpretation == "MONOCHROME1":
                img_array = np.amax(img_array) - img_array

            axes[i].imshow(img_array, cmap='gray')  # Menampilkan gambar DICOM dalam grayscale

        else:
            # Jika bukan file DICOM, membuka gambar biasa menggunakan PIL
            img = Image.open(img_path)
            axes[i].imshow(img)  # Menampilkan gambar dengan format yang sesuai

        axes[i].axis('off')  # Menonaktifkan axis pada gambar
        axes[i].set_title(image_file)  # Menampilkan nama file gambar sebagai judul

    # Menonaktifkan axis pada subplot yang tidak terpakai
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()  # Menyesuaikan layout agar gambar tidak saling bertumpukan
    plt.show()  # Menampilkan grid gambar


In [ ]:

folder_path = 'aneurysm'  # Mendefinisikan path folder tempat gambar 

# Membuat daftar file gambar yang memiliki ekstensi .png, .jpg, .jpeg, .bmp, .gif, atau .dcm (DICOM)
image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.dcm'))]

# Jika tidak ada gambar yang ditemukan, mencetak pesan error
if not image_files:
    print("Tidak ada gambar yang ditemukan dalam folder.")
else:
    n_cols = 5  # Menentukan jumlah kolom pada tampilan gambar
    n_rows = (len(image_files) + n_cols - 1) // n_cols  # Menentukan jumlah baris gambar berdasarkan jumlah file gambar

    # Membuat subplots untuk menampilkan gambar-gambar dalam grid
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5 * n_cols, 5 * n_rows))

    axes = axes.flatten()  # Mengubah array 2D axes menjadi 1D agar lebih mudah diakses

    # Menampilkan gambar satu per satu
    for i, image_file in enumerate(image_files):
        img_path = os.path.join(folder_path, image_file)  # Mendapatkan path lengkap untuk setiap gambar

        # Jika gambar adalah file DICOM, memprosesnya sebagai gambar medis
        if img_path.lower().endswith('.dcm'):
            dicom_data = pydicom.dcmread(img_path)  # Membaca file DICOM
            img_array = apply_voi_lut(dicom_data.pixel_array, dicom_data)  # Mengaplikasikan VOI LUT pada data piksel

            # Jika gambar DICOM dalam format MONOCHROME1, mengonversi gambar menjadi format yang benar
            if dicom_data.PhotometricInterpretation == "MONOCHROME1":
                img_array = np.amax(img_array) - img_array

            axes[i].imshow(img_array, cmap='gray')  # Menampilkan gambar DICOM dalam grayscale

        else:
            # Jika bukan file DICOM, membuka gambar biasa menggunakan PIL
            img = Image.open(img_path)
            axes[i].imshow(img)  # Menampilkan gambar dengan format yang sesuai

        axes[i].axis('off')  # Menonaktifkan axis pada gambar
        axes[i].set_title(image_file)  # Menampilkan nama file gambar sebagai judul

    # Menonaktifkan axis pada subplot yang tidak terpakai
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()  # Menyesuaikan layout agar gambar tidak saling bertumpukan
    plt.show()  # Menampilkan grid gambar


### 1. Handling Missing Data

In [ ]:
def handle_missing_image(image_path):
    try:
        img = Image.open(image_path).convert('RGB')
        return img
    except Exception:
        print(f"Tidak ada gambar yang rusak: {image_path}")
        return ImageOps.expand(Image.new('RGB', (64, 64), color='gray'), border=1)

# Daftar folder
folder_paths = ['tumor', 'cancer', 'aneurysm']

# Proses setiap gambar dalam folder
for folder in folder_paths:
    folder_path = os.path.join('brain_dataset', folder)  # Path lengkap ke folder
    if os.path.exists(folder_path):
        for image_file in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_file)
            try:
                img = handle_missing_image(image_path)  # Panggil fungsi untuk tiap gambar
                print(f"Gambar '{image_file}' di folder '{folder}' diproses dengan sukses.")
            except Exception as e:
                print(f"Error pada gambar '{image_file}' di folder '{folder}': {e}")
    else:
        print(f"Folder '{folder}' tidak ditemukan.")


### 2. Handling Outliers (Pixel Clipping/Smoothing)

In [ ]:
def smooth_image(image):
    # Menerapkan Gaussian Blur untuk smoothing
    return image.filter(ImageFilter.GaussianBlur(radius=1))

# Contoh penggunaan
img_smoothed = smooth_image(img)  # Menggunakan gambar dari langkah sebelumnya

# Menampilkan gambar di Notebook
plt.figure(figsize=(6, 6))
plt.imshow(img_smoothed)
plt.axis('off')  # Menghilangkan axis
plt.title("Gambar Setelah Smoothing (Gaussian Blur)")
plt.show()

### 3. Data Normalization

In [ ]:
import numpy as np

def normalize_image(image):
    # Konversi gambar menjadi array dan normalisasi
    image_array = np.array(image).astype('float32') / 255.0
    return image_array

# Contoh penggunaan
img_normalized = normalize_image(img_smoothed)  # Menggunakan gambar setelah smoothing
print(f"Array gambar setelah normalisasi:\n{img_normalized}")


### 4. Binning (Resolution Downsampling)

In [ ]:
# Definisikan path ke folder gambar
base_path = "brain_dataset"  # Ganti dengan path ke dataset Anda
folders = ["cancer", "tumor", "aneurysm"]  # Nama-nama folder

# Tentukan faktor binning
bin_factor = 2

# Loop untuk membaca gambar dari setiap folder
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    for image_file in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_file)
        
        # Pastikan hanya file gambar yang dibaca
        if image_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image = Image.open(image_path)
            
            # Jika gambar berupa NumPy array, konversikan menjadi PIL.Image
            if isinstance(image, np.ndarray):
                image = Image.fromarray(image)
            
            # Dapatkan ukuran asli gambar
            width, height = image.size

            # Hitung dimensi baru untuk binning
            new_width, new_height = width // bin_factor, height // bin_factor

            # Lakukan downsampling (resize gambar)
            downsampled_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)

            # Tampilkan gambar asli dan hasil downsampling
            plt.figure(figsize=(10, 5))
            
            plt.subplot(1, 2, 1)
            plt.title(f"Original Image - {folder}")
            plt.imshow(image, cmap="gray")
            plt.axis("off")
            
            plt.subplot(1, 2, 2)
            plt.title("Downsampled Image")
            plt.imshow(downsampled_image, cmap="gray")
            plt.axis("off")
            
            plt.show()

            # Untuk demonstrasi, cukup tampilkan satu gambar per folder
            break

### 5. Encoding (Image Augmentation)

In [ ]:
def augment_image(image):
    # Melakukan augmentasi gambar
    augmented_images = [
        image.transpose(Image.FLIP_LEFT_RIGHT),  # Flipping horizontal
        image.transpose(Image.FLIP_TOP_BOTTOM),  # Flipping vertikal
        image.rotate(90),  # Rotasi 90 derajat
        image.rotate(180)  # Rotasi 180 derajat
    ]
    return augmented_images

# Contoh penggunaan
augmented_images = augment_image(img_smoothed)

# Menampilkan gambar hasil augmentasi di Jupyter Notebook
plt.figure(figsize=(10, 6))
for i, aug_img in enumerate(augmented_images):
    plt.subplot(1, 4, i+1)
    plt.imshow(aug_img)
    plt.axis('off')
    plt.title(f'Augmentasi ke-{i+1}')

plt.tight_layout()
plt.show()

### 6. Grouping (Channel Separation)

In [ ]:
def split_channels(image):
    # Pisahkan saluran RGB
    r, g, b = image.split()
    return np.array(r), np.array(g), np.array(b)

# Contoh penggunaan
r, g, b = split_channels(img_smoothed)

print("Saluran R (Red):")
print(r)

print("Saluran G (Green):")
print(g)

print("Saluran B (Blue):")
print(b)


### Support Vector Machine (SVM) (ALGO KELOMPOK)

In [ ]:
# Model Machine Learning SVM
# Data Preprocessing
data = []  # List untuk menyimpan data gambar yang telah diproses
labels = []  # List untuk menyimpan label (jenis gambar) yang sesuai
for _, row in ct_brain_data.iterrows():  # Iterasi untuk setiap baris data pada DataFrame
    # Menentukan path gambar .jpg dan .dcm berdasarkan kolom pada DataFrame
    jpg_path = os.path.join(extracted_folder, 'files', row['jpg'].lstrip('/')) if 'jpg' in row else None
    dcm_path = os.path.join(extracted_folder, 'files', row['dcm'].lstrip('/')) if 'dcm' in row else None
    img_path = jpg_path if jpg_path and os.path.exists(jpg_path) else dcm_path  # Memilih gambar yang ada
    label = row['type']  # Menentukan label dari kolom 'type' pada DataFrame
    
    try:
        # Jika gambar ada, baca dan proses gambar
        if img_path and os.path.exists(img_path):
            if img_path.lower().endswith('.dcm'):
                # Proses gambar DICOM
                dicom_data = pydicom.dcmread(img_path)  # Membaca file DICOM
                img_array = apply_voi_lut(dicom_data.pixel_array, dicom_data)  # Mengaplikasikan LUT untuk DICOM
                if dicom_data.PhotometricInterpretation == "MONOCHROME1":
                    img_array = np.amax(img_array) - img_array  # Mengonversi gambar untuk format MONOCHROME1
            else:
                # Proses gambar biasa
                img = Image.open(img_path).convert("L")  # Mengonversi gambar ke grayscale
                img = img.resize(image_size)  # Mengubah ukuran gambar menjadi (64, 64)
                img_array = np.array(img)  # Mengubah gambar menjadi array numpy
            img_array = img_array.flatten() / 255.0  # Meratakan array gambar dan normalisasi
            data.append(img_array)  # Menambahkan array gambar ke data
            labels.append(label)  # Menambahkan label ke labels
    except Exception as e:
        print(f"Error processing {img_path}: {e}")  # Menangani error jika ada
    
# Preprocessing: Mengonversi data dan labels ke array numpy
data = np.array(data)
labels = np.array(labels)

# Encode Labels
label_encoder = LabelEncoder()  # Membuat objek LabelEncoder untuk mengonversi label kategori menjadi numerik
labels_encoded = label_encoder.fit_transform(labels)  # Mengonversi label kategori ke angka

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(data, labels_encoded, test_size=0.2, random_state=42)  # Membagi data menjadi set pelatihan dan pengujian

# Model Training with SVM
svm_model = SVC(kernel='linear', C=1, random_state=42)  # Membuat model SVM dengan kernel linear
svm_model.fit(X_train, y_train)  # Melatih model dengan data pelatihan

# Model Evaluation
y_pred = svm_model.predict(X_test)  # Memprediksi hasil untuk data pengujian
accuracy = accuracy_score(y_test, y_pred)  # Menghitung akurasi model
print(f"Test Accuracy: {accuracy:.2f}")  # Mencetak akurasi pada data pengujian
print("Classification Report:")  # Menampilkan laporan klasifikasi
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))  # Mencetak laporan klasifikasi


In [ ]:
# Confusion Matrix Visualization
cm = confusion_matrix(y_test, y_pred)  # Menghitung matriks kebingungannya antara label sebenarnya (y_test) dan label prediksi (y_pred)

# Membuat figure untuk plot dengan ukuran 8x6
plt.figure(figsize=(8, 6))

# Menampilkan heatmap dari matriks kebingungannya
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)

# Menambahkan label pada sumbu X dan Y
plt.xlabel("Predicted Labels")  # Label untuk sumbu X (label prediksi)
plt.ylabel("True Labels")  # Label untuk sumbu Y (label sebenarnya)

# Menambahkan judul pada plot
plt.title("Confusion Matrix - SVM")  # Judul untuk matriks kebingungannya

# Menampilkan plot
plt.show()


In [ ]:
# Visualisasi sampel gambar uji dengan label asli dan prediksi
num_images = min(8, len(X_test))  # Menentukan jumlah gambar yang akan ditampilkan, maksimal 8 gambar
fig, axes = plt.subplots(2, 4, figsize=(15, 8))  # Membuat grid 2x4 untuk menampilkan gambar
axes = axes.flatten()  # Mengubah array 2D axes menjadi 1D agar lebih mudah diakses

# Konversi label y_test dan y_pred ke bentuk yang dapat dibaca (dari kode ke label asli)
y_test_labels = label_encoder.inverse_transform(y_test)  # Mengonversi label sebenarnya ke label asli
y_pred_labels = label_encoder.inverse_transform(y_pred)  # Mengonversi prediksi label ke label asli

# Menampilkan gambar dan membandingkan label asli dengan prediksi
for i in range(num_images):
    img_array = X_test[i].reshape(image_size)  # Mengubah array gambar kembali ke ukuran 2D (misalnya 64x64)
    actual_label = y_test_labels[i]  # Label sebenarnya
    predicted_label = y_pred_labels[i]  # Label prediksi
    
    axes[i].imshow(img_array, cmap='gray')  # Menampilkan gambar dalam format grayscale
    
    # Tentukan warna judul berdasarkan apakah prediksi benar atau salah
    color = "green" if actual_label == predicted_label else "red"
    axes[i].set_title(f"Actual: {actual_label}\nPred: {predicted_label}", color=color, fontsize=10, pad=10)
    axes[i].axis('off')  # Menonaktifkan sumbu pada gambar

# Hilangkan sumbu yang tersisa jika ada
for j in range(num_images, len(axes)):
    axes[j].axis('off')  # Menonaktifkan sumbu pada subplot yang tidak digunakan

plt.suptitle("Sample Test Images with Actual and Predicted Labels", fontsize=14)  # Memberikan judul untuk seluruh figure
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Mengatur layout agar tidak saling tumpang tindih
plt.show()  # Menampilkan plot


### K-nearest neighbors (KNN)

In [ ]:
#Model Machine Learning KNN
data = []  
labels = []  


for _, row in ct_brain_data.iterrows():
    jpg_path = os.path.join(extracted_folder, 'files', row['jpg'].lstrip('/')) if 'jpg' in row else None
    dcm_path = os.path.join(extracted_folder, 'files', row['dcm'].lstrip('/')) if 'dcm' in row else None
    img_path = jpg_path if jpg_path and os.path.exists(jpg_path) else dcm_path  # Choose the available image
    label = row['type']  # Label

    try:
        if img_path and os.path.exists(img_path):
            if img_path.lower().endswith('.dcm'):
                dicom_data = pydicom.dcmread(img_path)  
                img_array = apply_voi_lut(dicom_data.pixel_array, dicom_data)  
                if dicom_data.PhotometricInterpretation == "MONOCHROME1":
                    img_array = np.amax(img_array) - img_array  
            else:
                img = Image.open(img_path).convert("L")  
                img = img.resize((64, 64))  
                img_array = np.array(img)  

            data.append(img_array.flatten())  
            labels.append(label)  

    except Exception as e:
        print(f"Error processing {img_path}: {e}")


X = np.array(data)
y = np.array(labels)


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


knn = KNeighborsClassifier(n_neighbors=5)  # You can change the number of neighbors (k)
knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)


print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn):.2f}")

In [ ]:
cm_knn = confusion_matrix(y_test, y_pred_knn)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title("Confusion Matrix - KNN")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
# Visualisasi sampel gambar uji dengan label asli dan prediksi
num_images = min(8, len(X_test))  # Menentukan jumlah gambar yang akan ditampilkan, maksimal 8 gambar
fig, axes = plt.subplots(2, 4, figsize=(15, 8))  # Membuat grid 2x4 untuk menampilkan gambar
axes = axes.flatten()  # Mengubah array 2D axes menjadi 1D agar lebih mudah diakses

# Konversi label y_test dan y_pred ke bentuk yang dapat dibaca (dari kode ke label asli)
y_test_labels = label_encoder.inverse_transform(y_test)  # Mengonversi label sebenarnya ke label asli
y_pred_labels = label_encoder.inverse_transform(y_pred_knn)  # Mengonversi prediksi label ke label asli

# Menampilkan gambar dan membandingkan label asli dengan prediksi
for i in range(num_images):
    img_array = X_test[i].reshape(image_size)  # Mengubah array gambar kembali ke ukuran 2D (misalnya 64x64)
    actual_label = y_test_labels[i]  # Label sebenarnya
    predicted_label = y_pred_labels[i]  # Label prediksi
    
    axes[i].imshow(img_array, cmap='gray')  # Menampilkan gambar dalam format grayscale
    
    # Tentukan warna judul berdasarkan apakah prediksi benar atau salah
    color = "green" if actual_label == predicted_label else "red"
    axes[i].set_title(f"Actual: {actual_label}\nPred: {predicted_label}", color=color, fontsize=10, pad=10)
    axes[i].axis('off')  # Menonaktifkan sumbu pada gambar

# Hilangkan sumbu yang tersisa jika ada
for j in range(num_images, len(axes)):
    axes[j].axis('off')  # Menonaktifkan sumbu pada subplot yang tidak digunakan

plt.suptitle("Sample Test Images with Actual and Predicted Labels", fontsize=14)  # Memberikan judul untuk seluruh figure
plt.tight_layout(rect=[0, 0.03, 1, 0.95])  # Mengatur layout agar tidak saling tumpang tindih
plt.show()  # Menampilkan plot

----

## <div align="center"> Comparison  </div>

Berdasarkan hasil perbandingan, kinerja SVM dan KNN menunjukkan performa yang identik dengan akurasi sebesar 0.98 untuk kedua model. Nilai precision, recall, dan F1-score pada masing-masing kelas juga sangat serupa. Pada SVM, kelas yang dianalisis menggunakan label deskriptif seperti aneurysm, cancer, dan tumor, sedangkan KNN menggunakan angka (0, 1, dan 2). Hasil makro rata-rata dan tertimbang untuk precision, recall, dan F1-score sama-sama mencapai 0.98, yang menegaskan performa konsisten antara kedua algoritma. Dengan demikian, kedua model ini bekerja sangat baik dalam klasifikasi dataset ini, dan pemilihan antara SVM atau KNN dapat didasarkan pada faktor lain seperti kompleksitas data, interpretabilitas model, dan efisiensi komputasi.

----

In [ ]:
studentName2 = "Marcell Nathan Santoso"
studentNIM2 = "00000092351"
studentClass2 = "EL"

In [ ]:
print("Name: \t\t{}".format(studentName2))
print("NIM: \t\t{}".format(studentNIM2))
print("student Class: \t\t{}".format(studentClass2))
print("End: \t\t{}".format(myDate))
print("Device ID: \t{}".format(myDevice))

----

## <div align="center"> Reference </div>

### Input Your Reference Here  (Jika ada):

----